In [293]:
#problems with model:
#nao diferencia entre dinheiro passivo e nao passivo
#complicado de perceber de onde para onde o dinheiro aparece
#complicado de perceber como o dinheiro desaparece

import datetime

from Google import Create_Service

CLIENT_SECRET_FILE = 'credentials.json'
API_NAME = 'sheets'
API_VERSION = 'v4'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

# https://developers.google.com/sheets/api/samples/data

spreadsheet_id = '15MQFom7ul8Lyw1knX9xiXd71QyHvZ67YyNXDTi2uPWs'#'15MQFom7ul8Lyw1knX9xiXd71QyHvZ67YyNXDTi2uPWs8'
sheet_id = '1739626240'

range_ = 'Transactions!I8:Y8'

request = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_, valueRenderOption='UNFORMATTED_VALUE')
response = request.execute()

range_ = 'Transactions!I1:Y1'
requestheaders = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_)#, valueRenderOption=value_render_option)
responseheaders = requestheaders.execute()


credentials.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
sheets service created successfully


In [331]:
#obtain initial snapshot - amounts per month
headers = responseheaders['values'][0]
values = response['values'][0]

accounts = {}

def CreateAccount(name,amount=0,desc='',annualrate=0, accounts={}):
    accounts[name]= {
   'Amount':amount}        
    return accounts
    
#populate with sheets
for h,v in zip(headers,values):
    #print(h,v)
    accounts = CreateAccount(name=h,amount=v)   

    #money i intend to spend every month
    monthlyspending=1400
    #money i intend to earn every month for nothing
    monthlypassive=600

###Alterar Variavel monthly spending em acima

In [332]:
ProgrammedTransactions

,From,To,Amount,When,Start,End,Desc,Unnamed: 7,This transactions has priority in the order they are
0,-,Moey,1310.7,*,*,*,Salario,NaN,NaN
1,-,Moey,1310.7,"6,12",*,*,Subsidio Natal,NaN,NaN
2,-,ActivoBank,300.0,"5,12",*,*,Natal e Anos,NaN,NaN
3,Moey,-,1000.0,*,06/2021,*,Quando Sair de Casa,NaN,NaN
4,Moey,Degiro Custody,800.0,*,*,*,Invest Degiro,NaN,NaN
5,Moey,Revolut Invest in EUR(1 Dollar = 0.88 EUR<),800.0,*,*,*,Invest Revolut,NaN,NaN
6,-,Paypal (EUR),40.0,*,*,*,Unity YT Medium ETC,NaN,NaN


In [333]:
#import programmables
import pandas as pd

# read data
ProgrammedTransactions = pd.read_csv("ProgrammedTransactions.csv")
ProgrammedAccounts = pd.read_csv("ProgrammedAccounts.csv")



def ProcessTransaction(tx,accounts):

    #from exists
    if tx['From']  not in accounts and tx['From'] != '-':
        print("INVALID Sender")
        return

    #to exist
    if tx['To']  not in accounts and tx['To'] != '-':
        print("INVALID Receiver")
        return

    #print(accounts[tx['To']]['Amount'])
    #print(tx['Amount'])

    #balanço inferior ah transaçao?
    if tx['From']!='-' and accounts[tx['From']]['Amount']<tx['Amount']:
        print("No Enough Balance")
        return


    #perform transaction
    if tx['From'] != '-':
        accounts[tx['From']]['Amount']=accounts[tx['From']]['Amount'] - tx['Amount']
    if tx['To'] != '-':
        accounts[tx['To']]['Amount']=accounts[tx['To']]['Amount'] + tx['Amount']
    
    print('Transaction\t{}\t->\t{}\t€{}\t{}'.format(tx['From'],tx['To'],tx['Amount'],tx['Desc']))




def shouldTransactionOccur(tx,curMon):

    start=tx["Start"]
    end=tx["End"]
    when=tx["When"]
    if(start=="*"):
        start = datetime.datetime(1,1,1)
    else:
        start= datetime.datetime.strptime(start, '%m/%Y')

    if(end=="*"):
        end = datetime.datetime(9999,1,1)
    else:
        end= datetime.datetime.strptime(end, '%m/%Y')
    #d.isoformat()
    
    if(when!="*"):
        when = str(when).split(',')
        #if month is not in specified months dont do it
        if(str(curMon.month) not in when):
            return
    
    
    

    is_between = start <= curMon <= end
    return is_between
        
def ProcessRates(acc,accounts,curMon):

    amnt = accounts[acc['Account']]['Amount']

    #if account has amount smaller than zero, skip 
    if(amnt<0):
        print("Negative Amount, cant apply interest rates")
        return
    
    #rate is zero
    if(acc['Rate']<=0):
        #print("Skipping")
        return

    if(amnt>acc['MaxAmountRate']):
        amnt=acc['MaxAmountRate']
        
    start=acc["Start"]
    end=acc["End"]
    #print(start,end)
    if(start=="*"):
        start = datetime.datetime(1,1,1)
    else:
        start= datetime.datetime.strptime(start, '%m/%Y')

    if(end=="*"):
        end = datetime.datetime(9999,1,1)
    else:
        end= datetime.datetime.strptime(end, '%m/%Y')
   

    is_between = start <= curMon <= end

    if is_between == False:
        return
    id acc['Compound']=='Y':
        rate = (pow(1+acc['Rate'],1/12)-1) 
        #was acc['Rate']/12
        #now is pow(1+r,1/12)-1
        #https://www.quora.com/How-do-you-convert-an-annual-interest-to-a-monthly-rate
        accounts[acc['Account']]['Amount'] = accounts[acc['Account']]['Amount'] + amnt*rate
    
    else:
     print("simple intererest")
     
    #print('We are the {} who say "{}!"'.format('knights', 'Ni'))
    print('Interest\t{}\t->\t{}\t€{}'.format(acc['Account'],acc['Account'],amnt* acc['Rate']/12))


In [343]:
r=0.1
r/12


0.008333333333333333

In [334]:
import datetime
import altair as alt
from dateutil.relativedelta import relativedelta


#qunado vais morrer - usada p simular a quantia que precisas para viver ate la
dayidie = datetime.datetime(2086,1,1)
curMon=datetime.datetime.now()

lifetime = dayidie-curMon
#months till i die
monthstolive = int(lifetime.days/30.2)

data={
    'moneyneededtolive':[],
    'moneyneededtolivewpassive':[],
    'networth':[],
    'x':[]
}

#years to graph
yearstocalculate = 30

#month id
seriesofmonths = range(1,monthstolive)

#for every month in the next  yearstocalculate
for i in seriesofmonths:
    
    #if(i==10):
    #    break
    
    if(i/12>yearstocalculate):
        break
    
    #ano - 24 corresponde  a minha idade
    data['x'].append(24+i/12)#i)
    
    curMon = curMon + relativedelta(months=1)
    print(curMon)
    #quantia pa viver
    data['moneyneededtolive'].append((monthlyspending)*(monthstolive-i))
    
    #counting passive - na pratica é como se eu em vez de gastasse, 1500, gastasse so 900€, isto e o poder do passive income
    data['moneyneededtolivewpassive'].append((monthlyspending-monthlypassive)*(monthstolive-i))
    
    #for every programed trnascction
    for index, tx in ProgrammedTransactions.iterrows():
        #checks if should be proefers
        
        if shouldTransactionOccur(tx,curMon):
            #performs it
            ProcessTransaction(tx,accounts)
          
    allmoneys=0
            
    #for every programed account
    for index, acc in ProgrammedAccounts.iterrows():
        #checks if should be proefers
        ProcessRates(acc,accounts,curMon)
        
        allmoneys= allmoneys + accounts[acc['Account']]['Amount']
            #performs it


  
    data['networth'].append(allmoneys)
    
        
    
import pandas as pd
    
#d = {'x': seriesofyears, 'Needed Money': moneyneededtolive,'Net Worth':networth}
#df = pd.DataFrame(data=d)
df=pd.DataFrame.from_dict(data)

dfmelted= df.melt('x', var_name='Series', value_name='y')

2021-02-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
Transaction	Moey	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€800.0	Invest Revolut
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.17235
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€38.35875
Interest	Mintos	->	Mintos	€1.6099000000000003
Interest	Bondora	->	Bondora	€66.48001875000001
Interest	Degiro Custody	->	Degiro Custody	€99.044175
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.09858333333333334
Interest	Bankinter	->	Bankinter	€20.833333333333332
Interest	XTB	->	XTB	€1.25
Interest	Trading 212	->	Trading 212	€3.9086250000000007
2021-03-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
Transaction	Moey	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€800.0	Invest Revolut
Transaction	-	->	Paypa

Interest	Degiro Custody	->	Degiro Custody	€234.74135793259208
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.10294547472756327
Interest	XTB	->	XTB	€1.3927104537882633
Interest	Trading 212	->	Trading 212	€4.354866317950524
2023-05-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	-	->	ActivoBank	€300.0	Natal e Anos
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.267251983332751
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€87.97378800745076
Interest	Mintos	->	Mintos	€1.683937172918113
Interest	Bondora	->	Bondora	€77.35079688001551
Interest	Degiro Custody	->	Degiro Custody	€242.5019181170865
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.10311705051877589
Interest	XTB	->	XTB	€1.3985134140123812
Interest	Trading 212	->	Trading 212	€4.373011

Interest	Revolut BitCoin	->	Revolut BitCoin	€0.1073217637011859
Interest	XTB	->	XTB	€1.5452752794750761
Interest	Trading 212	->	Trading 212	€4.831921271390621
2025-06-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	-	->	Moey	€1310.7	Subsidio Natal
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
Transaction	Moey	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€800.0	Invest Revolut
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.35704778306387
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€120.90889392655579
Interest	Mintos	->	Mintos	€1.7555225991023846
Interest	Bondora	->	Bondora	€88.99514772850308
Interest	Degiro Custody	->	Degiro Custody	€377.3676021414399
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.10750063330735453
Interest	XTB	->	XTB	€1.5517139264728887
Interest	Trading 212	->	Trading 212	€4.852

Interest	Trading 212	->	Trading 212	€5.565665211151317
2028-04-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.482210089679031
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€164.93403145422755
Interest	Mintos	->	Mintos	€1.8577871714412881
Interest	Bondora	->	Bondora	€107.6942255147802
Interest	Degiro Custody	->	Degiro Custody	€594.819860941702
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.11376287470314124
Interest	XTB	->	XTB	€1.7873470475117326
Interest	Trading 212	->	Trading 212	€5.5888554828644486
2028-05-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	-	->	ActivoBank	€300.0	Natal e Anos
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity 

No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.630212272427082
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€231.18709421602765
Interest	Mintos	->	Mintos	€1.9824470791017579
Interest	Bondora	->	Bondora	€134.0290112188053
Interest	Degiro Custody	->	Degiro Custody	€934.3113583915571
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.12139650985451376
Interest	XTB	->	XTB	€2.102011660756291
Interest	Trading 212	->	Trading 212	€6.572780262018861
2031-08-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.6340707826541045
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€232.3430296871078
Interest	Mintos	->	Mintos	€1.98575115756

...

Interest	Degiro Custody	->	Degiro Custody	€1875.891943088847
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.13686051110751124
Interest	XTB	->	XTB	€2.835651028788829
Interest	Trading 212	->	Trading 212	€8.866797201919814
2037-08-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€4.920502770492591
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€390.18584866176155
Interest	Mintos	->	Mintos	€2.2387045449876797
Interest	Bondora	->	Bondora	€201.85118297789214
Interest	Degiro Custody	->	Degiro Custody	€1895.9611326620134
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.1370886119593571
Interest	XTB	->	XTB	€2.847466241408782
Interest	Trading 212	->	Trading 212	€8.903742190261147
2037-09-28 14:13:30.600050
Transaction

Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€529.0557031152999
Interest	Mintos	->	Mintos	€2.420963250661449
Interest	Bondora	->	Bondora	€262.7399947059467
Interest	Degiro Custody	->	Degiro Custody	€2857.6847110025096
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.14824934910721677
Interest	XTB	->	XTB	€3.462033169521335
Interest	Trading 212	->	Trading 212	€10.825431517776293
2041-08-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.12122699086766
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€531.7009816308765
Interest	Mintos	->	Mintos	€2.424998189412552
Interest	Bondora	->	Bondora	€264.2179071761676
Interest	Degiro Custody	->	Degiro Custody	€2879.1173463350283
Interest	Revolut BitCoin	

Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.250816803449195
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€638.9831973656466
Interest	Mintos	->	Mintos	€2.549224403274467
Interest	Bondora	->	Bondora	€312.6396757103803
Interest	Degiro Custody	->	Degiro Custody	€3703.3010224886907
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.15610350896917508
Interest	XTB	->	XTB	€3.9383204564797865
Interest	Trading 212	->	Trading 212	€12.314734235366677
2044-03-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.255192484118736
Interest	Revolu

No Enough Balance
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.334581224973813
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€715.0518963201054
Interest	Mintos	->	Mintos	€2.6311722370303503
Interest	Bondora	->	Bondora	€347.7997447835177
Interest	Degiro Custody	->	Degiro Custody	€4326.29119625514
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.16112164090973094
Interest	XTB	->	XTB	€4.262076765676934
Interest	Trading 212	->	Trading 212	€13.327087838595245
2045-10-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.339026709327958
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€718.

Interest	Bni Europa Corr	->	Bni Europa Corr	€5.515320901461035
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€903.7695568884352
Interest	Mintos	->	Mintos	€2.8124068223527945
Interest	Bondora	->	Bondora	€435.283010633344
Interest	Degiro Custody	->	Degiro Custody	€5973.523186208894
Interest	Revolut BitCoin	->	Revolut BitCoin	€0.17221966534377717
Interest	XTB	->	XTB	€5.033305629760577
Interest	Trading 212	->	Trading 212	€15.73864337369841
2049-02-28 14:13:30.600050
Transaction	-	->	Moey	€1310.7	Salario
Transaction	Moey	->	-	€1000.0	Quando Sair de Casa
Transaction	Moey	->	Degiro Custody	€800.0	Invest Degiro
No Enough Balance
Transaction	-	->	Paypal (EUR)	€40.0	Unity YT Medium ETC
Interest	Bni Europa Corr	->	Bni Europa Corr	€5.519917002212252
Interest	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	->	Revolut Invest in EUR(1 Dollar = 0.88 EUR<)	€908.2884046728774
Interest	Mintos	->	Mintos	€2.817094167056716
Interest	Bondora	->	Bondora	€437.7

In [335]:
data['networth']

[57653.69906875001,
 59272.324531369355,
 60902.36011873373,
 62843.880058013354,
 64807.65907610986,
 65472.372403117246,
 66148.79577580749,
 66837.00544114085,
 67533.07815980092,
 68195.40454308824,
 70480.40895650558,
 71156.76952286882,
 71841.19989210748,
 72527.75880742191,
 73216.4604434351,
 74213.31907188043,
 76233.09406228169,
 76944.46522013827,
 77664.14789014608,
 78386.2025224427,
 79110.64501084755,
 79843.49135863896,
 82199.50267932398,
 82947.36053491385,
 83703.78203923623,
 84462.82942427206,
 85224.52037848828,
 86294.87271270787,
 88388.64936097374,
 89174.5337189185,
 89963.24474717207,
 90760.80153779431,
 91561.26831520442,
 92370.66477461715,
 94803.75574801679,
 95629.22654262575,
 96457.7980444114,
 97295.4912846188,
 98136.37244080097,
 99286.46317535652,
 101460.53030160611,
 102327.2611223638,
 103197.37853354805,
 104076.90559081864,
 104959.91051071261,
 105852.41700928773,
 108369.19396830598,
 109278.93077391257,
 110192.35242035527,
 111115.484076

In [336]:


source=dfmelted

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')



# The basic line
line = alt.Chart(source).mark_line().encode(
    x='x:Q',
    y='y:Q',
    color='Series:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='x:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'y:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='x:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)


alt.LayerChart(...)

In [337]:
dfmelted.to_csv("modelv2.csv")

In [338]:

first = df.iloc[[0]]['x'].values[0]
whatage=df[df['networth']>=df['moneyneededtolive']].iloc[[0]]['x'].values[0]

whatage2=df[df['networth']>=df['moneyneededtolivewpassive']].iloc[[0]]['x'].values[0]

inHowManyYears=whatage-first
inHowManyYears2=whatage2-first

inHowManyYears,inHowManyYears2

(22.83333333333334, 17.749999999999996)